In [9]:
import tensorflow as tf
from tensorflow import keras
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Flatten,Dense,BatchNormalization,Dropout,LeakyReLU,GlobalAveragePooling2D
from keras.optimizers import Adam,RMSprop
from keras.losses import BinaryCrossentropy
from keras.applications import ResNet50V2
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
import glob


image_size = 256

labels = ['NORMAL', 'PNEUMONIA']

train_path = "/kaggle/input/chest-xray-pneumonia/chest_xray/train/"
val_path = "/kaggle/input/chest-xray-pneumonia/chest_xray/val/"
test_path = "/kaggle/input/chest-xray-pneumonia/chest_xray/test/"


In [10]:
#data augment
train_val_generator = ImageDataGenerator(rescale=1./255,horizontal_flip=True,zoom_range=0.3)
test_generator = ImageDataGenerator(rescale=1./255)

train = train_val_generator.flow_from_directory(train_path,
                                               batch_size=128,
                                               target_size=(220,220),
                                               color_mode="rgb",
                                               class_mode="binary",
                                               shuffle=True,
                                               seed=42,
                                               subset="training")

val = train_val_generator.flow_from_directory(val_path,
                                               batch_size=4,
                                               target_size=(220,220),
                                               color_mode="rgb",
                                               class_mode="binary",
                                               shuffle=True,
                                               seed=42)

test = test_generator.flow_from_directory(test_path,
                                          batch_size=32,
                                          target_size=(220,220),
                                          color_mode="rgb",
                                          class_mode="binary")

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [11]:
resnet = ResNet50V2(weights = "imagenet", input_shape = (220,220,3), include_top = False)

In [12]:
for layer in resnet.layers:
    layer.trainable = False

In [13]:
model = Sequential()

model.add(resnet)
    
model.add(Flatten())

model.add(Dense(units = 128, activation = "relu"))
model.add(Dropout(0.25))

model.add(Dense(units = 1, activation = "sigmoid"))

In [14]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

In [15]:
hist = model.fit(train,validation_data=val,epochs=10)

Epoch 1/10
41/41 [==============================] - 114s 3s/step - loss: 2.5626 - accuracy: 0.8291 - val_loss: 1.2804 - val_accuracy: 0.8125
Epoch 2/10
41/41 [==============================] - 110s 3s/step - loss: 0.1621 - accuracy: 0.9423 - val_loss: 0.8959 - val_accuracy: 0.6875
Epoch 3/10
41/41 [==============================] - 111s 3s/step - loss: 0.1157 - accuracy: 0.9583 - val_loss: 1.1285 - val_accuracy: 0.8125
Epoch 4/10
41/41 [==============================] - 110s 3s/step - loss: 0.1156 - accuracy: 0.9594 - val_loss: 0.3572 - val_accuracy: 0.8750
Epoch 5/10
41/41 [==============================] - 111s 3s/step - loss: 0.0960 - accuracy: 0.9643 - val_loss: 0.6575 - val_accuracy: 0.8750
Epoch 6/10
41/41 [==============================] - 110s 3s/step - loss: 0.1040 - accuracy: 0.9647 - val_loss: 0.7383 - val_accuracy: 0.7500
Epoch 7/10
41/41 [==============================] - 110s 3s/step - loss: 0.0860 - accuracy: 0.9711 - val_loss: 0.2426 - val_accuracy: 0.8750
Epoch 8/10
41

In [16]:
test_results = model.evaluate(test)

20/20 [==============================] - 6s 317ms/step - loss: 0.2126 - accuracy: 0.9343
